# Running comparative analysis of multiple conditions

In order to compare multiple conditions and perform operations on them, we create a 'median reference' condition and compare all conditions in the dataset against the median condition. This then allows to do comparative analyses, such as heatmaps or general significance analyses (similar to ANOVA analyses on multiple conditions).


## Specifying input files

In order to run differential analyses you need two types of files:
* an input file from a proteomics search engine (e.g. DIA-NN, Spectronaut, AlphaPept etc.), or a pre-formatted file
* a sample mapping file that maps each sample to a condition (e.g.  sample 'brain_replicate_1' is mapped to condition 'brain'). In the GUI, there is some functionality to help create such a file

For the results file, AlphaQuant is compatible to the default output tables of most common proteomics search engines. Detailed specifications on which tables you need can be found 
in our  [README](https://github.com/MannLabs/alphaquant/blob/main/README.md#section-title).

The sample mapping file has to look as follows:


In [ ]:
INPUT_FILE = "./data/mouse_tissues/example_dataset_mouse_sn.tsv"
SAMPLEMAP_FILE = "./data/mouse_tissues/samplemap_200.tsv"
RESULTS_DIR = "./data/mouse_tissues/results_median_comparison"


In [ ]:
import pandas as pd

display(pd.read_csv(INPUT_FILE, sep="\t"))

In [ ]:
#displaying the samplemap file
import pandas as pd
display(pd.read_csv(SAMPLEMAP_FILE, sep='\t'))


## Run AlphaQuant in Median Comparison mode

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import alphaquant.run_pipeline as aqrunner

aqrunner.run_pipeline(input_file=INPUT_FILE, samplemap_file=SAMPLEMAP_FILE, multicond_median_analysis=True, results_dir=RESULTS_DIR)

## Inspect the results tables

There are four different main results tables written out to the directory:
1. `medianref_protein_alphaquant.tsv` -> protein quantities derived with AlphaQuant's cluster approach
2. `medianref_protein_avg.tsv` -> averaged protein quantities ('classic approach')
3. `medianref_proteoforms.tsv` -> quantities of all potential proteoforms
4. `medianref_peptides.tsv` -> peptide quantities, for expert users

In the following we will have a look at tables 1 and 3, which are likely to be the most relevant.

In [ ]:
import pandas as pd

df_protein = pd.read_csv(RESULTS_DIR + "/medianref_protein_alphaquant.tsv", sep='\t')
df_proteoform = pd.read_csv(RESULTS_DIR + "/medianref_proteoforms.tsv", sep='\t')

In [ ]:
display(df_protein)
display(df_proteoform)

The protein dataframe contains the protein quantities relative to the reference as well as a p_value for each protein which tests the null hypothesis that none of the conditions differs substantially from the median. In the example here all proteins are significant, which is not surprising, as we compare multiple tissues.\
The proteoform dataframe contains a bit more information than just protein quantities. A given protein can have multiple proteoform ids. A novel proteoform is defined if - for a given protein - one or more peptides exist that show a significantly altered regulatory profile as the other peptides that belong to the same protein, indicating a potential proteoform.

## Visualizing protein regulation using the protein results table

In [ ]:
#display dfs as clustermaps
import seaborn as sns
import numpy as np

nan_mask = df_protein.set_index('protein').drop(columns=["p_value"]).isna()

sns.clustermap(df_protein.set_index('protein').drop(columns=["p_value"]).replace(np.nan, 0), cmap="vlag", center=0, figsize=(12, 12), row_cluster=True, col_cluster=True, mask=nan_mask)

## Investigating proteoform profiles using the proteoform results table

We define the proteoform with the most consistent peptides as the 'reference proteoform'. The pearson correlation coefficient between the reference proteoform and each other proteoform is calculated and reported under the 'corr_to_ref' column. In other words, the 'corr_to_ref' column says how similar a proteoform behaves as compared to another proteoform. We use one of the AlphaQuant utility functions to filter for the interesting proteoforms, meaning those with low correlation compared to the reference proteoform.

In [ ]:
import alphaquant.multicond.multicond_utils as aq_multicond_utils

df_proteoform_low_corr = aq_multicond_utils.get_low_correlation_proteoform_df(proteoform_df=df_proteoform,
                                                                              max_correlation=0.7, keep_reference_proteoform=True)
#we specified that we always keep also the reference proteoform
display(df_proteoform_low_corr)

Now, we can use this dataframe to plot the profiles of the low correlation proteoforms and their reference proteoforms. For this we make use of the alphaquant plotting functions.

In [ ]:
import alphaquant.plotting.multicond as aq_plotting_multicond

In [ ]:
fig, ax = aq_plotting_multicond.plot_proteoform_intensity_profiles(proteoform_df=df_proteoform_low_corr)

